# Loads and Imports

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [2]:
#built-in
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import sys
import boto3

path = str(Path.home()) + '/rpi-canary-eval/temp'
git_path = str(Path.home()) + '/rpi-canary-eval'
sys.path.append(git_path + '/modules')


#custom
import data_process
from data_functions import make_date_to_MYR
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )

from path import path_dict, col_seq_cc, col_seq_elx, col_seq_cc_all

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

/home/nneehal/.local/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Download and Import Latent Space data and corresponding Original Data from S3

In [3]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/ldata/m6_propens_no_cluster_only_latent/treated0_l.p ../../temp/ldata/treated0_l.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/ldata/m6_propens_no_cluster_only_latent/match0_l.p ../../temp/ldata/match0_l.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/odata/m6_propens_no_cluster_only_latent/treated0_o.p ../../temp/odata/treated0_o.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/odata/m6_propens_no_cluster_only_latent/match0_o.p ../../temp/odata/match0_o.p

In [4]:
import pickle
with open(path+'/ldata_prop/match0_l.p', 'rb') as fp:
    controls_sick_latent = pickle.load(fp)
with open(path+'/ldata_prop/treated0_l.p', 'rb') as fp:
    treated_sick_latent = pickle.load(fp)
with open(path+'/odata_prop/match0_o.p', 'rb') as fp:
    controls_sick_original = pickle.load(fp)
with open(path+'/odata_prop/treated0_o.p', 'rb') as fp:
    treated_sick_original = pickle.load(fp)

# Sort and Filter

In [5]:
controls = controls_sick_latent[controls_sick_latent.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])
treated = treated_sick_latent[treated_sick_latent.PERS_ID>"0"].sort_values(by=["PERS_ID","MYR"])

In [6]:
controls_orig = controls_sick_original[controls_sick_original.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])
treated_orig = treated_sick_original[treated_sick_original.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])

## Evaluation Quality Match

## Metric = ASMD

In [7]:
def ASMD_calculate(cols, tf, cf):
    ASMD = pd.DataFrame()
    for col in cols:
        t_mean = np.mean(tf[col].values)
        c_mean = np.mean(cf[col].values)
        t_var = np.var(tf[col].values)
        c_var = np.var(cf[col].values)
        d = np.abs((t_mean-c_mean)/np.sqrt((t_var+c_var)/2))
        ASMD.loc[0, col] = d
    return ASMD

In [8]:
def ASMD_final(cols, cdf, tdf):
    cf = cdf[cols]
    tf = tdf[cols]
    ASMD = ASMD_calculate(cols, tf, cf)
    return ASMD

### Latent Space

In [9]:
latent_cols = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

In [10]:
ASMD = ASMD_final(latent_cols, controls, treated)
ASMD

x1        x2        x3        x4        x5        x6        x7        x8        x9       x10       x11       x12       x13       x14       x15       x16
0  0.098187  0.102289  0.105779  0.104653  0.115955  0.100954  0.103037  0.083685  0.101493  0.106536  0.105224  0.100528  0.102552  0.100179  0.101055  0.094503

### Original Space with George's Features only

In [11]:
george_cols = ['AGE_AT_MIDMONTH', 'MBR_GNDR', 'ALLOW_AMT','CC_TOBACCO',  'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID']

In [12]:
ASMD = ASMD_final(george_cols, controls_orig, treated_orig)
ASMD

AGE_AT_MIDMONTH  MBR_GNDR  ALLOW_AMT  CC_TOBACCO  CC_OBESITY  CC_HYPERTENSION  CC_HYPOTHYROID
0         0.117024    0.2593   0.050336    0.036589    0.309782          0.03868        0.012946

### Original Space with all the features

In [13]:
original_cols = controls_orig.columns.to_list()[2:]

In [14]:
ASMD = ASMD_final(original_cols, controls_orig, treated_orig)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


In [22]:
ASMD

AGE_AT_MIDMONTH  MBR_GNDR  MEDICAID_LOB  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS    CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR    CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0         0.117024    0.2593      0.321741   0.050336  0.014008  0.047853  0.04558      0.125046           0.187228  0.048303   0.028188              0.02197      0.022423   0.031711       0.115365   0.086426     0.02261  0.087771          0.060264     0.031066           0.003738            0.11223              0.034971  0.10521            0.049675       0.081493                 0.027859           0.014913     0.005475         0.091726   0.022387               0.026158     0.084011         0.009081     0.009292           0.041462          0.102227             NaN              0.020785                0.019069              0.030297                0.073925                    0.032186        0.017489             NaN          0.071214         0.068708           0.027204          0.03868        0.012946      0.046831     0.113473        0.024993       0.03727  0.067254        0.056584     0.131501            0.040951               0.059529                    NaN    0.309782   

   CC_OSTEOPOROSIS  CC_PERIPHERAL  CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CANCER   CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  
0         0.078423       0.133007         0.00861     0.070248            0.056867  0.025742          0.058735                0.084977        0.021483         0.014397   0.030819    0.036589             0.02587          0.042604